# Rayleigh-Bénard Convection in an annulus



In [1]:
import petsc4py
from petsc4py import PETSc

import underworld3 as uw
from underworld3.systems import Stokes
from underworld3 import function

import numpy as np
import sympy
import os

# to fix trame issue
import nest_asyncio
nest_asyncio.apply()

In [2]:
# The problem setup 

rayleigh_number_exponent = 5
rayleigh_number = pow(10,rayleigh_number_exponent)
resolution = 12

expt_name = f"Ra1e{rayleigh_number_exponent}_res{resolution}"
output_dir = os.path.join("output","annulus",f"Ra1e{rayleigh_number_exponent}")

os.makedirs(output_dir, exist_ok=True)


In [3]:
## Set up the mesh geometry / discretisation

r_o = 1
r_i = 0.5

meshball = uw.meshing.Annulus(
    radiusInner=r_i, 
    radiusOuter=r_o, 
    cellSize=1/resolution, 
    degree=1, 
    qdegree=3, 
)

x,y  = meshball.CoordinateSystem.X
r, th = meshball.CoordinateSystem.R
r_vector = meshball.CoordinateSystem.unit_e_0


In [4]:
v_soln = uw.discretisation.MeshVariable("U", meshball, meshball.dim, degree=2)
p_soln = uw.discretisation.MeshVariable("P", meshball, 1, degree=1)
t_soln = uw.discretisation.MeshVariable("T", meshball, 1, degree=3)
t_0 = uw.discretisation.MeshVariable("T0", meshball, 1, degree=2)

In [5]:
# passive_swarm = uw.swarm.Swarm(mesh=meshball)
# passive_swarm.populate(fill_param=3)

In [6]:
# Create solver to solver the momentum equation (Stokes flow)

stokes = Stokes(
    meshball, 
    velocityField=v_soln, 
    pressureField=p_soln, 
    solver_name="stokes",
)

stokes.constitutive_model = uw.constitutive_models.ViscousFlowModel
stokes.constitutive_model.Parameters.viscosity = 1.0

# Penalise flow crossing the boundary

stokes.add_natural_bc(10*rayleigh_number * v_soln.sym.dot(r_vector) * r_vector, "Upper")
stokes.add_natural_bc(10*rayleigh_number * v_soln.sym.dot(r_vector) * r_vector, "Lower")

stokes.bodyforce = r_vector * rayleigh_number * t_soln.sym[0] / (0.5) ** 3

In [7]:
# Create solver for the energy equation (Advection-Diffusion of temperature)

adv_diff = uw.systems.AdvDiffusion(
    meshball,
    u_Field=t_soln,
    V_fn=v_soln,
    solver_name="adv_diff",
    verbose=False,
)

adv_diff.constitutive_model = uw.constitutive_models.DiffusionModel
adv_diff.constitutive_model.Parameters.diffusivity = 1

## Boundary conditions for this solver

adv_diff.add_dirichlet_bc(1.0, "Lower")
adv_diff.add_dirichlet_bc(0.0, "Upper")

In [8]:
# The advection / diffusion equation is an initial value problem
# We set this up to have 

abs_r = sympy.sqrt(meshball.rvec.dot(meshball.rvec))
init_t = 0.05 * sympy.sin(2.0 * th) * sympy.sin(np.pi * (r - r_i) / (r_o - r_i)) + (
    r_o - r
) / (r_o - r_i)


with meshball.access(t_0, t_soln):
    t_0.data[...] = uw.function.evaluate(init_t, t_0.coords).reshape(-1, 1)
    t_soln.data[...] = t_0.data[...]

In [9]:
# Solution strategy: solve for the velocity field, then update the temperature field.
# First we check this works for a tiny timestep, later we will repeat this to 
# model the passage of time.

stokes.solve()
adv_diff.solve(timestep=0.00001 * stokes.estimate_dt())

0 - j_distance[0:50] - 5.3489839850766374e-05 / 1.5012066991297082e-05
 [4.31637037e-05 4.31637037e-05 4.81889683e-05 4.81889683e-05
 4.68779697e-05 4.35125271e-05 4.47084546e-05 4.58042361e-05
 4.85094293e-05 4.57367125e-05 5.34898399e-05 4.48170326e-05
 4.70773655e-05 4.49808310e-05 4.58570620e-05 4.66415187e-05
 4.74930714e-05 4.90065220e-05 4.79222221e-05 4.72377631e-05
 4.19825361e-05 5.07023201e-05 4.68779697e-05 4.35125271e-05
 4.47084546e-05 4.58042361e-05 4.85094293e-05 4.57367125e-05
 5.34898399e-05 4.48170326e-05 4.70773655e-05 4.49808310e-05
 4.58570620e-05 4.66415187e-05 4.74930714e-05 4.90065220e-05
 4.79222221e-05 4.72377631e-05 4.19825361e-05 5.07023201e-05
 4.76910037e-05 4.67674841e-05 4.78746718e-05 4.77873039e-05
 4.76987988e-05 4.17023143e-05 4.10875844e-05 4.63304513e-05
 4.77476478e-05 4.68057115e-05]
1 - j_distance[0:50] - 0.042176245198611156 / 0.028584817530839978
 [0.0332796  0.0332796  0.03963347 0.03963347 0.03874906 0.03927608
 0.03927429 0.03937435 0.0412

In [10]:
def plot_T_mesh(filename):
    if uw.mpi.size == 1:
        
        import pyvista as pv
        import underworld3.visualisation as vis

        pvmesh = vis.mesh_to_pv_mesh(meshball)
        pvmesh.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh, t_soln.sym)
        pvmesh.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh, stokes.u.sym)

        tpoints = vis.meshVariable_to_pv_cloud(t_soln)
        tpoints.point_data["T"] = vis.scalar_fn_to_pv_points(tpoints, t_soln.sym)
        tpoint_cloud = pv.PolyData(tpoints)

        velocity_points = vis.meshVariable_to_pv_cloud(stokes.u)
        velocity_points.point_data["V"] = vis.vector_fn_to_pv_points(velocity_points, stokes.u.sym)

        # point sources at cell centres
        cpoints = pvmesh.cell_centers().points
        cpoint_cloud = pv.PolyData(cpoints)
    
        pvstream = pvmesh.streamlines_from_source(
            cpoint_cloud,
            vectors="V",
            integrator_type=2,
            integration_direction="both",
            compute_vorticity=False,
            max_steps=1000,
            max_time=0.1,
            initial_step_length=0.01,
            surface_streamlines=True,
            )

        pl = pv.Plotter(window_size=(750, 750))

        pl.add_mesh(
                    pvmesh,
                    cmap="RdBu_r",
                    edge_color="Black",
                    show_edges=True,
                    scalars="T",
                    use_transparency=False,
                    opacity=1.0,
                    show_scalar_bar=False,
                )

        pl.add_mesh(pvstream, 
                    opacity=0.75,             
                    show_scalar_bar=False,
)

        pl.screenshot(
            filename="{}.png".format(filename),
            window_size=(1280, 1280),
            return_img=False,
        )


In [11]:
timestep = 0
elapsed_time=0.0

In [12]:
# Convection model / update in time

output = os.path.join(output_dir, expt_name)

for step in range(0, 200):
    stokes.solve()
    delta_t = 3.0 * stokes.estimate_dt()
    adv_diff.solve(timestep=delta_t)

    # stats then loop
    tstats = t_soln.stats()

    if uw.mpi.rank == 0:
        print(f"Timestep {timestep}, dt {delta_t}, t {elapsed_time}")

    # plot_T_mesh(filename=f"{output}_step_{timestep}")

    # meshball.petsc_save_checkpoint(index=step, 
    #                                meshVars=[v_soln, t_soln], 
    #                                outputPath=output_dir)

    meshball.write_timestep(filename="annulus_ckpt",
                            index=timestep,
                            outputPath=output_dir,
                            meshVars=[v_soln, p_soln, t_soln],
                        )

    timestep += 1
    elapsed_time += delta_t


0 - j_distance[0:50] - 5.3489839850766374e-05 / 1.5012066991297082e-05
 [4.31637037e-05 4.31637037e-05 4.81889683e-05 4.81889683e-05
 4.68779697e-05 4.35125271e-05 4.47084546e-05 4.58042361e-05
 4.85094293e-05 4.57367125e-05 5.34898399e-05 4.48170326e-05
 4.70773655e-05 4.49808310e-05 4.58570620e-05 4.66415187e-05
 4.74930714e-05 4.90065220e-05 4.79222221e-05 4.72377631e-05
 4.19825361e-05 5.07023201e-05 4.68779697e-05 4.35125271e-05
 4.47084546e-05 4.58042361e-05 4.85094293e-05 4.57367125e-05
 5.34898399e-05 4.48170326e-05 4.70773655e-05 4.49808310e-05
 4.58570620e-05 4.66415187e-05 4.74930714e-05 4.90065220e-05
 4.79222221e-05 4.72377631e-05 4.19825361e-05 5.07023201e-05
 4.76910037e-05 4.67674841e-05 4.78746718e-05 4.77873039e-05
 4.76987988e-05 4.17023143e-05 4.10875844e-05 4.63304513e-05
 4.77476478e-05 4.68057115e-05]
1 - j_distance[0:50] - 0.042176245198611156 / 0.028584817530839978
 [0.0332796  0.0332796  0.03963347 0.03963347 0.03874906 0.03927608
 0.03927429 0.03937435 0.0412

KeyboardInterrupt: 

In [ ]:
if uw.mpi.size == 1:
    
    import pyvista as pv
    import underworld3.visualisation as vis

    pvmesh = vis.mesh_to_pv_mesh(meshball)
    pvmesh.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh, t_soln.sym)
    pvmesh.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh, v_soln.sym)

    points = vis.meshVariable_to_pv_cloud(t_soln)
    points.point_data["T"] = vis.scalar_fn_to_pv_points(points, t_soln.sym)
    point_cloud = pv.PolyData(points)

    velocity_points = vis.meshVariable_to_pv_cloud(stokes.u)
    velocity_points.point_data["V"] = vis.vector_fn_to_pv_points(velocity_points, stokes.u.sym)

    # point sources at cell centres
    cpoints = pvmesh.cell_centers().points
    cpoint_cloud = pv.PolyData(cpoints)

    pvstream = pvmesh.streamlines_from_source(
            cpoint_cloud,
            vectors="V",
            integrator_type=2,
            integration_direction="both",
            compute_vorticity=False,
            max_steps=1000,
            max_time=0.1,
            initial_step_length=0.01,
            surface_streamlines=True,
            )

    pl = pv.Plotter(window_size=(750, 750))


    # pl.add_arrows(velocity_points.points, velocity_points.point_data["V"], mag=0.00005, opacity=0.75)
    # pl.add_arrows(arrow_loc2, arrow_length2, mag=1.0e-1)

    # pl.add_points(
    #     point_cloud,
    #     cmap="coolwarm",
    #     render_points_as_spheres=True,
    #     point_size=7.5,
    #     opacity=0.75,
    # )

    pl.add_mesh(pvstream, opacity=0.5)
    pl.add_mesh(pvmesh, scalars="T", cmap="RdBu_r", opacity=1)

    pl.show(cpos="xy")